In [64]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import PIL
import os
import easyocr
import openpyxl

# Этап первый - ищем якорь на картинке

<p>dzen - якорь - день/дней <br>
телега - якорь - err/vr <br>
vk - якорь подписчик (выше приоритет) - участник (ниже приоритет) <br>
youtube - якоря:  подписчики + подробнее (выше приоритет),время просмотра</p>

In [79]:
text = pd.DataFrame(columns=['Файл', 'Успешная загрузка', 'Метрика 1', 'Метрика 2'])

In [80]:
#всё в одной функции, чтобы не плодить сущности - создали Reader и гоняем его в рамках одной картинки

#триггер удачного нахождения якоря
load = 0

#надо ещё доработать эту функцию, мне кажется за счёт
#улучшения качества фото можно улучшить предикты модели
def cut_at_anchor(img_path, soc_net):
    #открываем изображение с помощью Pillow
    try:
        image = Image.open(img_path)
    except FileNotFoundError:
        print('Файл повреждён')

    #обрабатываем фото для лучшего распознавания
    #есть ещё над чем поработать
    #но уже хорошо работает
    result = image.convert('L')
    image = result.filter(ImageFilter.SHARPEN)
    gray = np.asarray(image)

    #создаём экземпляр сети
    if(soc_net!='tg'):
        reader = easyocr.Reader(['ru'])             
    else:
        reader = easyocr.Reader(['en'])
        
    #вытаскиваем текст из изображения
    result = reader.readtext(gray, batch_size=512, min_size=5)

    #считываем размеры фото
    width, height = gray.size

#Этап второй: ищем якорь и его позицию
#если якорь найти не удалось - выдаём ошибку ничего не записываем в pandas
    if(soc_net=='vk'):
        word_to_find = 'подписчик'
        word_to_find2 = 'участник'
        for r in result:
            text = r[1]
            if word_to_find in text or word_to_find2 in text:
                load = 1
                #записываем позицию левого верхнего угла якоря
                top_pos = r[0][0]
                #обрезаем фото так: влево на 30 пикселей, вправо на 150 (сама надпись + растояние до цифры) 
                #и на высоту 50 (можно поиграть для точности)
                cropped_image = image.crop((top_pos[0]-30, top_pos[1], top_pos[0]+150, top_pos[1]+50))
    elif(soc_net=='youtube'):
        word_to_find = 'подписчик'
        word_to_find2 = 'просмотра'
        word_to_find3 = 'подробнее'
        #подписчики => вверх и вниз обрезаем по 50 пикслей, если есть слово подробнее - оно становится главным якорем, над ним цифра
        #значит режем вверх
    elif(soc_net=='dzen'):
        word_to_find = 'день'
        word_to_find2 = 'дней'
        for r in result:
            text = r[1]
            if word_to_find in text or word_to_find2 in text:
                load = 1
                #из левого верхнего угла якоря идём до конца фото (width) и на 50 пикселей вниз
                #2 цифра - наша
                cropped_image = image.crop((top_pos[0], top_pos[1], width, top_pos[1]+50))
    else:
        word_to_find = 'ERR'
        word_to_find2 = 'VR'
        #тут другая логика если нашёл ERR - обрезать слево, если нашёл VR - обрезать внизу
        for r in result:
            text = r[1]
            #заглушки пока
            if word_to_find in text:
                load = 1
                print('')
                #обрезать изображение слево на 50 пикселей
            elif word_to_find2 in text:
                load = 1
                print('')
                #обрезать изображение вниза на 50 пикселей

    result = reader.readtext(cropped_image, batch_size=512, min_size=5)
    #тут нужна функция очистки строки, чтобы на выходе была только метрика
    return result

In [78]:
#тут будет разный паттер в зависимости от соцсети
data = text.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    data.to_excel(writer, sheet_name='Telegram')
    data.to_excel(writer, sheet_name='VK')
    data.to_excel(writer, sheet_name='Youtube')
    data.to_excel(writer, sheet_name='Dzen')

In [69]:
# Путь к папке, в которой находятся папки с изображениями
root_folder = "temp"

In [81]:
# Обход всех папок и файлов внутри главной папки
for root, dirs, files in os.walk(root_folder):
    for file in files:
    # Проверка, что файл - изображение (замените на поддерживаемые форматы по своему усмотрению)
        if file.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')):
            # Получение полного пути к файлу
            file_path = os.path.join(root, file)
            # Обработка изображения с помощью функции process_image
            #вызываемм функцию которая вернёт метрики (если их 2, если нет метрику + пустую строку)
            metricOne, metricTwo = cut_at_anchor(file_path, soc_net)
            
            if(load==1):
                print(str(file)+'\t'+'Успешно загружен')
                text.loc[len(text)] = [file, string, 'Успешно загружен', '-']
            elif(load==0):
                print(str(file)+'\t'+'Требуется ручная проверка')
                text.loc[len(text)] = [file, string, 'Требуется ручная проверка', '-']

NameError: name 'soc_net' is not defined

In [36]:
text.to_excel('tg.xlsx')

In [38]:
df = pd.read_excel('tg.xlsx')

In [40]:
df.to_csv('tg.csv')